In [186]:
import pandas as pd
from sqlalchemy import create_engine,text
import os
import json
import vertexai
from vertexai.generative_models import GenerativeModel
import time

### Vertex config

In [194]:
VERTEX_PROJECT_ID = os.getenv("VERTEX_PROJECT_ID")
vertex_region = "us-west4"

### Get all video data from database

In [187]:
db_string = 'sqlite:///../db/youtube.db'
# Create a engine
engine = create_engine(db_string)
# Create connection
conn = engine.connect()

In [188]:
# Get all video data
query = text("SELECT * FROM video")
video_df = pd.read_sql_query(query, conn)

### Remove shorts and very long format videos

In [189]:
def filter_videos_by_duration(df, min_duration=60, max_duration=1800):
    """
    Filter videos DataFrame by duration within a specified range.
    
    Parameters:
        df (DataFrame): The DataFrame containing video data.
        min_duration (int): Minimum duration in seconds. Default is 60.
        max_duration (int): Maximum duration in seconds. Default is 1800.
        
    Returns:
        DataFrame: Filtered DataFrame containing videos with duration within the specified range.
    """
    return df[(df['duration'] >= min_duration) & (df['duration'] <= max_duration)]

In [190]:
videos_for_labelling_df = filter_videos_by_duration(video_df)

### Create traning dataset

In [259]:
# Create training dataset
videos_for_labelling_df = videos_for_labelling_df.head(100) # defines the size of the training dataset

### Create Gemini prompt

In [192]:
# Adjust display options to prevent truncation
pd.set_option('display.max_colwidth', None)

In [258]:
# Write prompt string
prompt_str = """You are a classifier, and I want you to classify each of the following data science youtube video titles based on three overarching categories, video_type, video_topic, coding_language

"video_type": the options for video type are: 

"Tutorial",
"Project",
"News",
"Tips",
"Challenge",
"Career Advice",
"Podcast/Interview

"video_topic": the options for video topic are: 

"Statistics and Probability",
"Machine Learning / AI",
"Data Wrangling",
"Data Visualization",
"Data Mining",
"Software Engineering",
"Ethics and Privacy",
"Cloud Computing",
"Resume Building",
"Job Search Strategies",
"Interview Techniques",
"Career Development Paths",
"Balancing Work and Life",
"Business Acumen",

"technologies": also pull out any software/coding_language/packages as a list called 'technologies'. This should not include anything other than software / coding_language / packages though

From now on, return the output as a regular Python dict object. Your entire response should be a single line. Do not wrap strings with '\' There should be items within this for each video, for the classifications.

The response should look like (for example):

{
  "videos": [
    {
      "video_id": "YdWkUdMxMvM",
      "video_title": "Career Change to Code - The Complete Guide",
      "video_info": {
        "video_type": "Career Advice",
        "video_topic": "Software Engineering",
        "technologies": []
      }
    },
    {
      "video_id": "5rNk7m_zlAg",
      "video_title": "Spring Boot & Spring Data JPA – Complete Course",
      "video_info": {
        "video_type": "Tutorial",
        "video_topic": "Software Engineering",
        "technologies": ["Spring", "Spring Boot"]
      }
    },
  ]
}

There should be no trailing comma after the last item in the "videos" list!

The video ids and titles are below:

"""

### Function to generate respone from Gemini

In [247]:
def generate_response(project_id: str, location: str, prompt) -> str:
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location)
    # Load the model
    multimodal_model = GenerativeModel("gemini-1.0-pro")
    # Query the model
    response = multimodal_model.generate_content(
        [
            prompt
        ]
    )
    return response.text

### Generate responses

In [256]:
# Get total number of videos
total_videos = len(videos_for_labelling_df)

# Define chunk size
chunk_size = 50

# Initialize response list
responses = []

# Loop through the DataFrame in chunks of 50
for chunk_start in range(0, total_videos, chunk_size):
    # Get chunk of videos
    videos_chunk = videos_for_labelling_df.iloc[chunk_start:chunk_start + chunk_size]

    # Initialize prompt
    prompt = prompt_str

    # Add video information to prompt
    for i, (_, row) in enumerate(videos_chunk.iterrows(), start=1):
        video_title = row['video_title']
        video_id = row['video_id']
        prompt += f'\nvideo_id : {video_id}, video_title: {video_title}'

    # Generate response for the chunk
    response_text = generate_response(VERTEX_PROJECT_ID, vertex_region, prompt)

    video_info_dict = json.loads(response_text)

    # Map video_type, video_topic, and technologies to the existing DataFrame
    for video in video_info_dict['videos']:
        video_id = video['video_id']
        video_type = video['video_info']['video_type']
        video_topic = video['video_info']['video_topic']
        technologies = ', '.join(video['video_info']['technologies'])
        print(f"video id: {video_id}")
        print(f"video type: {video_type}")
        print(f"video topic: {video_topic}")
        print(f"video tech: {technologies}")
    
        # Update the DataFrame with the extracted information
        videos_for_labelling_df.loc[videos_for_labelling_df['video_id'] == video_id, 'video_type'] = video_type
        videos_for_labelling_df.loc[videos_for_labelling_df['video_id'] == video_id, 'video_topic'] = video_topic
        videos_for_labelling_df.loc[videos_for_labelling_df['video_id'] == video_id, 'technologies'] = technologies

    chunk = chunk_start + chunk_size
    print(f'Completed {chunk} rows...')

    # Add a delay before processing the next chunk
    time.sleep(10)

video id: 9He4UBLyk8Y
video type: Tips
video topic: Software Engineering
video tech: 
video id: ypNKKYUJE5o
video type: Tutorial
video topic: Software Engineering
video tech: JavaScript
video id: D6Xj_W4leu8
video type: Project
video topic: Software Engineering
video tech: OpenAI API, ChatGPT
video id: xZbU6bCZFYo
video type: News
video topic: Software Engineering
video tech: 
video id: flpmSXVTqBI
video type: Tutorial
video topic: Software Engineering
video tech: Java, JUnit 5


In [257]:
videos_for_labelling_df.head()

,channel_id,video_id,video_title,description,tags,published,view_count,like_count,favourite_count,comment_count,duration,definition,caption,category_id,video_type,video_topic,technologies
57,UC8butISFwT-Wl7EV0hUK0BQ,9He4UBLyk8Y,Front End Developer Roadmap 2024,Learn what technologies you should learn first to become a front end web developer.\nWatch the full learning front end learning path: https://www.youtube.com/playlist?list=PLWKjhJtqVAbmMuZ3saqRIBimAKIMYkt0E\n\n✏️ This video was developed by @beau \n\n🎉 Thanks to our Champion and Sponsor supporters:\n👾 davthecoder\n👾 jedi-or-sith\n👾 南宮千影\n👾 Agustín Kussrow\n👾 Nattira Maneerat\n👾 Heather Wcislo\n👾 Serhiy Kalinets\n👾 Justin Hual\n👾 Otis Morgan \n👾 Oscar Rahnama\n\n--\n\nLearn to code for free and get a developer job: https://www.freecodecamp.org\n\nRead hundreds of articles on programming: https://freecodecamp.org/news,,2023-10-19 14:18:42.000000,507722.0,17091.0,0,493.0,729,hd,false,27,Tips,Software Engineering,
134,UC8butISFwT-Wl7EV0hUK0BQ,ypNKKYUJE5o,JavaScript Security Vulnerabilities Tutorial – With Code Examples,Learn about 10 security vulnerabilities every JavaScript developer should know. First try to find the vulnerabilities in the different code examples. Then learn how to fix the issues.\n\n✏️ Brandon from Semgrep developed this course.\n\n🔗 Learn more about Semgrep: https://go.semgrep.dev/scan-in-1-minute-for-free\n🏗 Semgrep provided a grant to make this course possible.\n\n\n\n🎉 Thanks to our Champion and Sponsor supporters:\n👾 davthecoder\n👾 jedi-or-sith\n👾 南宮千影\n👾 Agustín Kussrow\n👾 Nattira Maneerat\n👾 Heather Wcislo\n👾 Serhiy Kalinets\n👾 Justin Hual\n👾 Otis Morgan\n\n--\n\nLearn to code for free and get a developer job: https://www.freecodecamp.org\n\nRead hundreds of articles on programming: https://freecodecamp.org/news,,2023-05-16 14:37:07.000000,62016.0,2625.0,0,71.0,1505,hd,true,27,Tutorial,Software Engineering,JavaScript
158,UC8butISFwT-Wl7EV0hUK0BQ,D6Xj_W4leu8,Use ChatGPT to Build a RegEx Generator – OpenAI API Low Code Course,"Learn how to build a dashboard that generates RegEx using the OpenAI API and a low code tool called Retool. The dashboard allows users to input a string, write a description of the desired RegEx, and generate the code to transform the string accordingly. \n\n💻 You can find the iframe code here: https://github.com/kubowania/regex-openai\n\n✏️ This course was created by @AniaKubow .\n\n🏗 Retool provided a grant to make this course possible. Learn more about Retool: https://retool.com\n\n⭐️ Contents ⭐️\n(0:00) Introduction\n(2:36) Writing our own regex code\n(5:06) Creating the header element\n(8:11) Creating the text area\n(11:35) How to create a custom code editor\n(16:32) Creating the new button\n(19:08) Setting up the sidebar\n(20:58) Configuring the open api api\n(22:51) The body of the example example\n(24:58) Setting up the code\n\n🎉 Thanks to our Champion and Sponsor supporters:\n👾 Nattira Maneerat\n👾 Heather Wcislo\n👾 Serhiy Kalinets\n👾 Erdeniz Unvan\n👾 Justin Hual\n👾 Agustín Kussrow\n👾 Otis Morgan\n\n--\n\nLearn to code for free and get a developer job: https://www.freecodecamp.org\n\nRead hundreds of articles on programming: https://freecodecamp.org/news",,2023-03-30 13:32:31.000000,102762.0,2133.0,0,82.0,1792,hd,true,27,Project,Software Engineering,"OpenAI API, ChatGPT"
489,UC8butISFwT-Wl7EV0hUK0BQ,xZbU6bCZFYo,freeCodeCamp.org Curriculum Expansion: Math + Machine Learning + Data Science,"Support our campaign here: https://www.freecodecamp.org/news/p/4476d664-eb83-47c9-8328-903a78865c8f#the-2021-data-science-curriculum-pledge-drive\n\nView the Curriculum Design Sheets (these are very much a work in progress): https://docs.google.com/spreadsheets/d/1KXEJvgnAziiePMp2Ovf_ssDqCFFWYuycXWBNp91k3qM/edit?usp=sharing\n\nIf you're a math / CS professor or Data Science practitioner, we'd appreciate your feedback / ideas for this curriculum: https://docs.google.com/forms/d/e/1FAIpQLScLiGgmYh2DxdTmPxS141J5knuOtcYN9ulOtBwsRbkuVwXRRQ/

In [ ]:
videos_for_labelling_df.to_csv('/training_dataset/labelled_video_dataset.csv')